In [9]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
import json

# Scrape with json

### Get name, brand, category, lowest price and average price if there's more than 1 supplier for each product inside Bebidas catalog


In [4]:
url='https://www.cotabest.com.br/catalogo/search/?q=&category_id=null&category[]=bebidas&page={}&order_by=-relevance'
#sumarle todas las paginas que tenga

In [5]:
df = pd.DataFrame(columns = ['Name','Brand','Category','Lowest Price','Avg Price'])

In [6]:
for nro_pagina in range(1,6):
    url_pagina = url.format(nro_pagina)
    r = req.get(url_pagina)
    contenido = r.json()
    products = contenido['results']
    for product in products:
        multipliers = product['multipliers']
        prices = []
        for dic in multipliers:
            prices.append(float(dic['unit_price']))
        avg = sum(prices)/len(prices)
        df.loc[len(df.index)] = [product['full_display'], product['brand'], product['category'], min(prices), avg]


In [5]:
df

,Name,Brand,Category,Lowest Price,Avg Price
0,Refrigerante lata 350ml - Coca-Cola,Coca-Cola,Refrigerantes,2.59,3.194000
1,Refrigerante Pet 2 Litros - Coca-Cola,Coca-Cola,Refrigerantes,7.41,9.261667
2,Refrigerante Guaraná lata 350ml - Antarctica,Antarctica,Refrigerantes,2.33,2.710000
3,Whisky Red Label garrafa 1 Litro - Johnnie Walker,Johnnie Walker,Whisky,79.45,95.258182
4,Cerveja long neck 330ml - Heineken,Heineken,Cerveja,5.29,5.990000
...,...,...,...,...,...
95,Isotônico sabor Maracujá pet 500ml - Gatorade,Gatorade,Isotônicos e Hidrotônicos,3.89,4.145000
96,Refrigerante Limão pet 2 Litros - Dolly,Dolly,Refrigerantes,4.15,4.395000
97,Refrigerante pet 2 Litros - Pepsi,Pepsi,Refrigerantes,6.34,6.340000
98,Água de Coco tetra pak 1 Litro - Sococo,Sococo,Água de Coco,6.43,8.126667


# Scrape with html

### Get item number, description, brand, pack size, unit and weight for each product on the catalog


In [10]:
url = 'http://www.universityfoods.info/catalog/allitems?offset=0&limit=250&col=item_no&dir=ASC&terms=&queryCol='

In [12]:
data = req.get(url,"html")
soup = BeautifulSoup(data.content,'html.parser')
tables = soup.find_all('table')
table = tables[4]
table_info = {}
for row_id, row in enumerate(table.find_all('tr')):
    col = []
    for val in row.find_all('td'):
        val = val.text
        val = val.replace('\n','').strip()
        #if val:
        col.append(val)
    table_info['row_{}'.format(row_id+1)] = col
    
table_info.pop('row_1',None)
table_info.pop('row_2',None)
table_info.pop('row_3',None)
table_info.pop('row_4',None)
table_info.pop('row_5',None)


['1', '----------', '', '', 'EACH', '']

In [13]:
df = pd.DataFrame(columns = ['ItemID','Description','Brand','Pack size','Unit','Weight','Category'])

In [14]:
table_info

{'row_6': ['CHICKEN-FRESH'],
 'row_7': ['10003PC',
  'CHICKEN WOG 2.75#-3# FRESH HD',
  'PLATINUM BRAND',
  'PC',
  'PC.',
  '2.7500*'],
 'row_8': ['CHICKEN-FROZEN'],
 'row_9': ['10005',
  'CHICKEN BRST 7 OZ B/L S/L SPLT IQF',
  'AGROSUPER',
  '4/10#',
  'CASE',
  '40.0000'],
 'row_10': ['10005PC',
  'CHICKEN BRST 7 OZ B/L S/L SPLT IQF',
  'AGROSUPER',
  '10#',
  'PC.',
  '10.0000'],
 'row_11': ['10006',
  'CHICKEN BRST 8 OZ B/L S/L SPLT IQF',
  'AGROSUPER',
  '4/10#',
  'CASE',
  '40.0000'],
 'row_12': ['10011',
  'CHICKEN LEG MEAT B/L S/L FRZ 4/10#',
  'PACKER',
  '4/10#',
  'CASE',
  '40.0000'],
 'row_13': ['CHICKEN-FRESH'],
 'row_14': ['10022',
  'CHICKEN BRST 7 OZ B/L S/L DBL FRS',
  'PLATINUM BRAND',
  '24/7 OZ',
  'CASE',
  '10.0000*'],
 'row_15': ['CHICKEN-FROZEN'],
 'row_16': ['10029',
  'CHICKEN BRST B/IN YELLOW 40# FRZ',
  'PACKER',
  '40#',
  'CASE',
  '40.0000'],
 'row_17': ['CHICKEN-CUT'],
 'row_18': ['10033',
  'CHICKEN BRST FAJITA STRIP RAW 10#',
  'PLATINUM BRAND',
  '

In [17]:
categoria = ""
i = 0
for clave in table_info:
    if len(table_info[clave]) > 1:
        df.loc[i,'ItemID'] = table_info[clave][0]
        df.loc[i,'Description'] = table_info[clave][1]
        df.loc[i,'Brand'] = table_info[clave][2]
        df.loc[i,'Pack size'] = table_info[clave][3]
        df.loc[i,'Unit'] = table_info[clave][4]
        df.loc[i,'Weight'] = table_info[clave][5]
        df.loc[i,'Category'] = categoria
        i+=1
    else:
        categoria = table_info[clave][0]
        
        
    

In [18]:
df

,ItemID,Description,Brand,Pack size,Unit,Weight,Category
0,10003PC,CHICKEN WOG 2.75#-3# FRESH HD,PLATINUM BRAND,PC,PC.,2.7500*,CHICKEN-FRESH
1,10005,CHICKEN BRST 7 OZ B/L S/L SPLT IQF,AGROSUPER,4/10#,CASE,40.0000,CHICKEN-FROZEN
2,10005PC,CHICKEN BRST 7 OZ B/L S/L SPLT IQF,AGROSUPER,10#,PC.,10.0000,CHICKEN-FROZEN
3,10006,CHICKEN BRST 8 OZ B/L S/L SPLT IQF,AGROSUPER,4/10#,CASE,40.0000,CHICKEN-FROZEN
4,10011,CHICKEN LEG MEAT B/L S/L FRZ 4/10#,PACKER,4/10#,CASE,40.0000,CHICKEN-FROZEN
...,...,...,...,...,...,...,...
242,13157,SAUCE HORSERADISH WHT 4/1 GAL,TULKOFF,4/1 GAL,CASE,,SAUCE
243,13158,MUSTARD YELLOW 12/12 OZ FRENCH'S,FRENCH'S,12/12 OZ,CASE,,CONDIMENTS
244,1315913159PC,MUSTARD YELLOW 4/1 GAL MOREHOUSE MUSTARD YELLO...,MOREHOUSEMOREHOUSE,4/1 GAL1/1 GAL,CASEGAL.,,CONDIMENTS
245,13168,VEGAN SOYRIZO NON GMO 4/2.5# 607,EL BURRITO,4/2.5#,CASE,,VEGAN
